## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import API key
from config2 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(r"C:\Users\test adobe\World_Weather_Analysis\World_Weather_Analysis\Challenge\Weather_Database\WeatherPy_Database.csv")
del city_data_df['Unnamed: 0']
pd.options.display.float_format ='{:,.2f}'.format
city_data_df.head()

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Vaini,TO,2022-02-06 19:09:25,-21.20,-175.20,75.36,94,100,10.36,light rain
1,Mataura,NZ,2022-02-06 19:11:48,-46.19,168.86,43.48,90,10,0.92,clear sky
2,Lagoa,PT,2022-02-06 19:08:46,39.05,-27.98,60.19,85,16,12.15,few clouds
3,Ushuaia,AR,2022-02-06 19:11:49,-54.80,-68.30,55.81,81,100,8.32,overcast clouds
4,Lenger,KZ,2022-02-06 19:11:49,42.18,69.88,29.64,97,100,8.97,snow


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the min temp in degrees Fahrenheit you would like to experience during your trip?"))
max_temp = float(input("What is the max temp in degrees Fahrenheit you would like to experiece during your trip"))

What is the min temp in degrees Fahrenheit you would like to experience during your trip?50
What is the max temp in degrees Fahrenheit you would like to experiece during your trip75


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]

preferred_cities_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,Lagoa,PT,2022-02-06 19:08:46,39.05,-27.98,60.19,85,16,12.15,few clouds
3,Ushuaia,AR,2022-02-06 19:11:49,-54.80,-68.30,55.81,81,100,8.32,overcast clouds
8,East London,ZA,2022-02-06 19:11:08,-33.02,27.91,74.82,82,1,22.03,clear sky
10,Punta Arenas,CL,2022-02-06 19:11:52,-53.15,-70.92,69.91,52,75,16.11,broken clouds
18,Pinheiro Machado,BR,2022-02-06 19:11:55,-31.58,-53.38,69.06,63,98,11.81,overcast clouds
23,New Norfolk,AU,2022-02-06 19:09:28,-42.78,147.06,52.72,75,4,1.95,clear sky
26,Kerrville,US,2022-02-06 19:08:58,30.05,-99.14,57.45,35,0,4.00,clear sky
29,Gazipur,BD,2022-02-06 19:11:59,24.00,90.42,64.36,72,40,0.00,haze
30,Cape Town,ZA,2022-02-06 19:08:47,-33.93,18.42,69.10,73,15,1.01,few clouds
31,Hilo,US,2022-02-06 19:10:53,19.73,-155.09,71.26,91,100,5.75,heavy intensity rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City                   203
Country                196
Date                   203
Lat                    203
Lng                    203
Max Temp               203
Humidity               203
Cloudiness             203
Wind Speed             203
Current Description    203
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna(subset = ["Country"], inplace=True)
preferred_cities_df.count()

City                   196
Country                196
Date                   196
Lat                    196
Lng                    196
Max Temp               196
Humidity               196
Cloudiness             196
Wind Speed             196
Current Description    196
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Lagoa,PT,60.19,few clouds,39.05,-27.98,
3,Ushuaia,AR,55.81,overcast clouds,-54.80,-68.30,
8,East London,ZA,74.82,clear sky,-33.02,27.91,
10,Punta Arenas,CL,69.91,broken clouds,-53.15,-70.92,
18,Pinheiro Machado,BR,69.06,overcast clouds,-31.58,-53.38,
23,New Norfolk,AU,52.72,clear sky,-42.78,147.06,
26,Kerrville,US,57.45,clear sky,30.05,-99.14,
29,Gazipur,BD,64.36,haze,24.00,90.42,
30,Cape Town,ZA,69.10,few clouds,-33.93,18.42,
31,Hilo,US,71.26,heavy intensity rain,19.73,-155.09,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()  
        
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping.")
        

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [13]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna(subset = ["Hotel Name"], inplace=True)
hotel_df.count()
hotel_df.head(20)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Lagoa,PT,60.19,few clouds,39.05,-27.98,Casa Das Faias
3,Ushuaia,AR,55.81,overcast clouds,-54.80,-68.30,Albatross Hotel
8,East London,ZA,74.82,clear sky,-33.02,27.91,Tu Casa
10,Punta Arenas,CL,69.91,broken clouds,-53.15,-70.92,Hotel Hain
18,Pinheiro Machado,BR,69.06,overcast clouds,-31.58,-53.38,Hotel e Restaurante Quitandinha
23,New Norfolk,AU,52.72,clear sky,-42.78,147.06,The Shingles Riverside Cottages
26,Kerrville,US,57.45,clear sky,30.05,-99.14,Inn Of The Hills Hotel & Conference Center
29,Gazipur,BD,64.36,haze,24.00,90.42,Murshida & Zakir's Nest
30,Cape Town,ZA,69.10,few clouds,-33.93,18.42,Southern Sun Waterfront Cape Town
31,Hilo,US,71.26,heavy intensity rain,19.73,-155.09,Hilo Hawaiian Hotel


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Temperature</dt><dd>{Max Temp} °F</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))